In [3]:
import os
import pandas as pd
os.getcwd()


'C:\\Users\\eunjin\\financeData'

In [8]:
path = './open'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼양홀딩스,000070,KOSPI
1,하이트진로,000080,KOSPI
2,유한양행,000100,KOSPI
3,CJ대한통운,000120,KOSPI
4,두산,000150,KOSPI
...,...,...,...
342,카카오게임즈,293490,KOSDAQ
343,레몬,294140,KOSDAQ
344,에이비엘바이오,298380,KOSDAQ
345,박셀바이오,323990,KOSDAQ


In [60]:
#각각 21년 11월, 21년 3월 상장 21년 5월 상장
stock_list=stock_list.drop(stock_list.loc[stock_list['종목코드']=='028150',].index) 
stock_list=stock_list.drop(stock_list.loc[stock_list['종목코드']=='031390',].index)
stock_list=stock_list.drop(stock_list.loc[stock_list['종목코드']=='036490',].index)
stock_list=stock_list.drop(stock_list.loc[stock_list['종목명']=='DL이앤씨',].index)
stock_list

,종목명,종목코드,상장시장
0,삼양홀딩스,000070,KOSPI
1,하이트진로,000080,KOSPI
2,유한양행,000100,KOSPI
3,CJ대한통운,000120,KOSPI
4,두산,000150,KOSPI
...,...,...,...
342,카카오게임즈,293490,KOSDAQ
343,레몬,294140,KOSDAQ
344,에이비엘바이오,298380,KOSDAQ
345,박셀바이오,323990,KOSDAQ


In [10]:
import FinanceDataReader as fdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pmdarima.arima import ndiffs
import pmdarima as pm

In [14]:
def forecast_one_step():
    fc, conf_int = model.predict(n_periods=1 # 한 스텝씩!
        , return_conf_int=True)              # 신뢰구간 출력
    return (
        fc.tolist()[0],
        np.asarray(conf_int).tolist()[0]
    )

In [16]:
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [74]:
#많은 종목 돌려보기 #예측할 날짜 : 11월 1일~5일 종료 
start_date = '2021-01-01'
end_date = '2021-11-05'
result = pd.DataFrame(columns = {"test", "pred"})
result
for i in range(0,len(stock_list)): #len(stock_list)
    try:
        sample_code = stock_list.loc[i,'종목코드']
        print("종목 진행 중", stock_list.iloc[i,0])
        stock = fdr.DataReader(sample_code,start_date, end_date)
        y_train = stock['Close'][:int(len(stock)-5)]  #train할 앞에 날짜들
        y_test = stock['Close'][int(len(stock)-5):] #test할 맨 뒤 일주일( 5일 )
        #차분이 필요한지, 몇 차 차분이 최선인지 파악하려면 ndiff 함수 이용
        kpss_diffs = ndiffs(y_train, alpha=0.05, test='kpss', max_d=6)
        adf_diffs = ndiffs(y_train, alpha=0.05, test='adf', max_d=6)
        n_diffs = max(adf_diffs, kpss_diffs)
    #     print(f"추정된 차수 d = {n_diffs}")
        model = pm.auto_arima(y = y_train        # array형태 시계열 자료
                          , d = n_diffs           # 차분 차수, ndiffs 결과!
                          , start_p = 0 #AR(p)찾을 범위 0~3
                          , max_p = 3   
                          , start_q = 0 #AR(q) 찾을 범위 0~3
                          , max_q = 3   
                          , m = 1       #계절적 차분? -> X 4면 분기별 12면 월별임.
                          , seasonal = False # 계절성 ARIMA가 아니라서.
                          , stepwise = True #최적의 모수를 찾기 위해 힌드만-칸다카르 알고리즘을 사용할지? false면 모든 모수 조합
                          , trace=False #true면 stepwise모델 적합할 때 마다 결과 프린트
                          )
        y_predict = model.predict(n_periods=len(y_test)) 
        y_predict = pd.DataFrame(y_predict,index = y_test.index,columns=['Prediction'])

        forecasts = []
        y_pred = []
        pred_upper = []
        pred_lower = []

        for new_ob in y_test:
            fc, conf = forecast_one_step()
            y_pred.append(fc)
            pred_upper.append(conf[1])
            pred_lower.append(conf[0])

            ## 모형 업데이트 !
            model.update(new_ob)

        #붙이는 방법 :: e = np.vstack((a,b))
        tmp = pd.DataFrame({"test": y_test, "pred": y_pred})
        result = pd.DataFrame(np.vstack((result, tmp)))

    #     from plotly.subplots import make_subplots
    #     import plotly.graph_objects as go

    #     fig = go.Figure([
    #         # 훈련 데이터-------------------------------------------------------
    #         go.Scatter(x = y_train.index, y = y_train, name = "Train", mode = 'lines'
    #                   ,line=dict(color = 'royalblue'))
    #         # 테스트 데이터------------------------------------------------------
    #         , go.Scatter(x = y_test.index, y = y_test, name = "Test", mode = 'lines'
    #                     ,line = dict(color = 'rgba(0,0,30,0.5)'))
    #         # 예측값-----------------------------------------------------------
    #         , go.Scatter(x = y_test.index, y = y_pred, name = "Prediction", mode = 'lines'
    #                          ,line = dict(color = 'red', dash = 'dot', width=3))

    #         # 신뢰 구간---------------------------------------------------------
    #         , go.Scatter(x = y_test.index.tolist() + y_test.index[::-1].tolist() 
    #                     ,y = pred_upper + pred_lower[::-1] ## 상위 신뢰 구간 -> 하위 신뢰 구간 역순으로
    #                     ,fill='toself'
    #                     ,fillcolor='rgba(0,0,30,0.1)'
    #                     ,line=dict(color='rgba(0,0,0,0)')
    #                     ,hoverinfo="skip"
    #                     ,showlegend=False)
    #     ])

    #     fig.update_layout(height=400, width=1000, title_text="ARIMA(최적)모형")
    #     fig.show()
    
    except:
        print(print("종목 에러 발생", stock_list.iloc[i,0]))

종목 진행 중 삼양홀딩스
종목 진행 중 하이트진로
종목 진행 중 유한양행
종목 진행 중 CJ대한통운
종목 진행 중 두산
종목 진행 중 DL
종목 진행 중 한국앤컴퍼니
종목 진행 중 기아
종목 진행 중 SK하이닉스
종목 진행 중 영풍
종목 진행 중 현대건설
종목 진행 중 삼성화재
종목 진행 중 한화
종목 진행 중 DB하이텍
종목 진행 중 CJ
종목 진행 중 JW중외제약
종목 진행 중 LG상사
종목 진행 중 동국제강
종목 진행 중 대한전선
종목 진행 중 현대해상
종목 진행 중 대상
종목 진행 중 SK네트웍스
종목 진행 중 오리온홀딩스
종목 진행 중 넥센타이어
종목 진행 중 KCC
종목 진행 중 아모레G
종목 진행 중 부광약품
종목 진행 중 대웅
종목 진행 중 삼양식품
종목 진행 중 태광산업
종목 진행 중 쌍용C&E
종목 진행 중 대한항공
종목 진행 중 영진약품
종목 진행 중 LG
종목 진행 중 포스코케미칼
종목 진행 중 보령제약
종목 진행 중 롯데정밀화학
종목 진행 중 현대제철
종목 진행 중 신세계
종목 진행 중 농심
종목 진행 중 세방전지
종목 진행 중 효성
종목 진행 중 롯데지주
종목 진행 중 녹십자홀딩스
종목 진행 중 롯데칠성
종목 진행 중 현대차
종목 진행 중 현대그린푸드
종목 진행 중 POSCO
종목 진행 중 DB손해보험
종목 진행 중 삼성전자
종목 진행 중 NH투자증권
종목 진행 중 동원산업
종목 진행 중 SK디스커버리
종목 진행 중 LS
종목 진행 중 녹십자
종목 진행 중 GS건설
종목 진행 중 삼성SDI
종목 진행 중 대한유화
종목 진행 중 미래에셋증권
종목 진행 중 GS리테일
종목 진행 중 오뚜기
종목 진행 중 일양약품
종목 진행 중 F&F 홀딩스
종목 진행 중 메리츠증권
종목 진행 중 호텔신라
종목 진행 중 한미사이언스
종목 진행 중 삼성전기
종목 진행 중 한샘
종목 진행 중 한올바이오파마
종목 진행 중 한국조선해양
종목 진행 중 한화솔루션
종목 진행 중 OCI
종목 진행 중 LS ELECTRIC
종목 진행 중 고려아연
종목 진행 중 삼성중공업


In [75]:
result

,0,1
0,106000,103000
1,104500,106000
2,105000,104500
3,105500,105000
4,105500,105500
...,...,...
1690,87900,79800
1691,86400,87900
1692,89000,86400
1693,91200,89000


In [76]:
score = NMAE(result.iloc[:,0], result.iloc[:,1])*100
print(f"평가 산식 NMAE*100: {score: .3f}")

평가 산식 NMAE*100:  1.665
